In [ ]:
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install pypianoroll

Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 155685 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.5) ...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.6 MB 6.8 MB/s 
     |████████████████████████████████| 51 kB 4.2 MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591955 sha256=3d7f4d0a96031e672daa490e56353af04a66ad2319ea62f05ac40033e26c8622
  Stored in directory: /root/.cache/pip/wheels/ad/74/7c/a06473ca8dcb63efb9

In [ ]:
import pypianoroll
from google.colab import drive
import os
import numpy as np
from datetime import datetime
import glob
import pandas as pd
import random
from collections import Counter
from google.colab import drive
import torch
import os
import pandas as pd
import gc
from sklearn.model_selection import train_test_split
device = 'cpu'
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:

def create_parts(mid):
  parts = {'Piano': None, 'Piano Right': None, 'Piano Left': None, 'Violin1': None, 'Violin2': None, 'Viola': None, 'Cello': None}

  programs = {'Piano': None, 'Piano Right': None, 'Piano Left': None, 'Violin1': None, 'Violin2': None, 'Viola': None, 'Cello': None}

  to_normalize = {
  'Right': 'Piano Right',
  'Left': 'Piano Left',
  'Grand Piano': 'Piano',
  'Violino I': 'Violin1',
  'Violino II': 'Violin2',
  '1st Violin': 'Violin1',
  '2nd Violin': 'Violin2',
  'Solo Cello': 'Cello',
  'Violoncello': 'Cello',
  'Cello Pizz': 'Cello',
  'Six Suites for Solo Cello': 'Cello',
  'Violin 1': 'Violin1',
  'Violin 2': 'Violin2',
  'Piano RH': 'Piano Right',
  'Piano LH': 'Piano Left',
  'Piano RH (Playback)': 'Piano Right',
  'Piano LH (Playback)': 'Piano Left',
  'Violino I.': 'Violin1',
  'Viola.': 'Viola',
  'Violin 1 arco': 'Violin1',
  'Cello arco': 'Cello',
  'Cello pizz': 'Cello',
  'Violino II.': 'Violin2',
  'Violin2 Pizz': 'Violin2',
  'Violin 2 pizz': 'Violin2',
  'Violin 2 arco': 'Violin2',
  'Cello (Playback)': 'Cello',
  'Viola pizz': 'Viola',
  'Viola Arco': 'Viola',
  'Violincello': 'Cello',
  'Violoncello.': 'Cello',
  'Violin1 Sub': 'Violin1',
  'Viola arco': 'Viola',
  'Viola Pizz': 'Viola',
  'Violin1 Pizz': 'Violin1',
  'Viola Sub': 'Viola',
  'Violin Pizz': 'Violin1',
  'Violin (Playback)': 'Violin1',
  'Viola (Playback)': 'Viola',
  'Acoustic Grand Piano': 'Piano',
  'Pianoforte': 'Piano',
  'Violin': 'Violin1',
  }

  to_ignore = [
      'Instrument17', 'Instrument18', 'Instrument19', 
      '--------------------------------------', 'Johann Sebastian Bach  (1685-1750)'
  ]

  for track in mid.tracks:
    if track.name not in to_ignore:
      if track.name in to_normalize.keys():
        normalized_name = to_normalize[track.name]
        if normalized_name in parts.keys() and parts[normalized_name] is None:
          parts[normalized_name] = track.pianoroll
          programs[normalized_name] = track.program
      elif track.name in parts.keys() and parts[track.name] is None:
        parts[track.name] = track.pianoroll
        programs[track.name] = track.program
      if track.pianoroll.shape[0] > 0:
        empty_array = np.zeros_like(track.pianoroll)

  for k,v in parts.items():
    if v is None:
      parts[k] = empty_array.copy()

  return parts, programs

In [ ]:
input_path = ''
raw_path = os.path.join(input_path, '00_raw')
preprocess_path = os.path.join(input_path, '01_preprocessed')
preparation_path = os.path.join(input_path, '02_preparation')

preprocess_df = pd.read_csv(os.path.join(preprocess_path, 'preprocessed_dataset.csv'))
mid_files_to_use = preprocess_df.mid_name.to_list()

all_composer_path = glob.glob(f"{raw_path}/*")

all_parts = []
all_programs = []

for composer_path in all_composer_path:
    composer_mid_files_path = glob.glob(f"{composer_path}/*.mid")
    for mid_file_path in composer_mid_files_path:
      try:
        if mid_file_path.split('/')[-1] in mid_files_to_use: #and len(all_parts) < 3
          mid = pypianoroll.read(os.path.join(mid_file_path))
          parts, programs = create_parts(mid)
          tensor_parts = torch.tensor([parts['Piano'], parts['Piano Right'], parts['Piano Left'], parts['Violin1'], parts['Violin2'], parts['Viola'], parts['Cello']])
          all_parts.append(tensor_parts)
          all_programs.append(programs)
      except Exception as e:
        print(e)
        print('problem', mid_file_path) 

del preprocess_df
del mid_files_to_use
del all_composer_path
gc.collect()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
/usr/local/lib/python3.7/dist-packages/pretty_midi/pretty_midi.py:101: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)


0

In [ ]:

data_concat = torch.hstack(all_parts)
del (all_parts)
gc.collect()
data_concat = data_concat[:, :, 24:96]
gc.collect()


0

In [ ]:

data_bin = data_concat.clone()
del (data_concat)
gc.collect()


22

In [ ]:
data_bin[data_bin > 0.2] = 1.0
data_bin = data_bin.type(torch.float16)


In [ ]:
# gan tensor
torch.save(data_bin, os.path.join(preparation_path, 'tensor.pt'))
